In [1]:
import numpy as np

import torch
from torch import optim

import random
from copy import deepcopy

from utils import get_data, ndcg, recall
from model import VAE

import easydict

In [2]:
args = easydict.EasyDict({
    'dataset': "output",
    'hidden_dim': 600,
    'latent_dim': 200,
    'batch_size': 500,
    'beta': None,
    'gamma': 0.005,
    'lr': 5e-4,
    'n_epochs': 50,
    'n_enc_epochs': 3,
    'n_dec_epochs': 3,
    'n_dec_epochs': 1,
    'not_alternating': False,
})

In [3]:
seed = 1337
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [4]:
device = torch.device("cuda:0")

In [5]:
data = get_data(args.dataset)
train_data, valid_in_data, valid_out_data, test_in_data, test_out_data = data

In [6]:
class Batch:
    def __init__(self, device, idx, data_in, data_out=None):
        self._device = device
        self._idx = idx
        self._data_in = data_in
        self._data_out = data_out
    
    def get_idx(self):
        return self._idx
    
    def get_idx_to_dev(self):
        return torch.LongTensor(self.get_idx()).to(self._device)
        
    def get_ratings(self, is_out=False):
        data = self._data_out if is_out else self._data_in
        return data[self._idx]
    
    def get_ratings_to_dev(self, is_out=False):
        return torch.Tensor(
            self.get_ratings(is_out).toarray()
        ).to(self._device)

In [7]:
def generate(batch_size, device, data_in, data_out=None, shuffle=False, samples_perc_per_epoch=1):
    assert 0 < samples_perc_per_epoch <= 1
    
    total_samples = data_in.shape[0]
    samples_per_epoch = int(total_samples * samples_perc_per_epoch)
    
    if shuffle:
        idxlist = np.arange(total_samples)
        np.random.shuffle(idxlist)
        idxlist = idxlist[:samples_per_epoch]
    else:
        idxlist = np.arange(samples_per_epoch)
    
    for st_idx in range(0, samples_per_epoch, batch_size):
        end_idx = min(st_idx + batch_size, samples_per_epoch)
        idx = idxlist[st_idx:end_idx]

        yield Batch(device, idx, data_in, data_out)

In [8]:
def evaluate(model, data_in, data_out, metrics, samples_perc_per_epoch=1, batch_size=500):
    metrics = deepcopy(metrics)
    model.eval()
    
    for m in metrics:
        m['score'] = []
    
    for batch in generate(batch_size=batch_size,
                          device=device,
                          data_in=data_in,
                          data_out=data_out,
                          samples_perc_per_epoch=samples_perc_per_epoch
                         ):
        
        ratings_in = batch.get_ratings_to_dev()
        ratings_out = batch.get_ratings(is_out=True)
    
        ratings_pred = model(ratings_in, calculate_loss=False).cpu().detach().numpy()
        
        if not (data_in is data_out):
            ratings_pred[batch.get_ratings().nonzero()] = -np.inf
            
        for m in metrics:
            m['score'].append(m['metric'](ratings_pred, ratings_out, k=m['k']))

    for m in metrics:
        m['score'] = np.concatenate(m['score']).mean()
        
    return [x['score'] for x in metrics]

In [9]:
def run(model, opts, train_data, batch_size, n_epochs, beta, gamma, dropout_rate):
    model.train()
    for epoch in range(n_epochs):
        for batch in generate(batch_size=batch_size, device=device, data_in=train_data, shuffle=True):
            ratings = batch.get_ratings_to_dev()

            for optimizer in opts:
                optimizer.zero_grad()
                
            _, loss = model(ratings, beta=beta, gamma=gamma, dropout_rate=dropout_rate)
            loss.backward()
            
            for optimizer in opts:
                optimizer.step()

In [10]:
model_kwargs = {
    'hidden_dim': args.hidden_dim,
    'latent_dim': args.latent_dim,
    'input_dim': train_data.shape[1]
}
metrics = [{'metric': ndcg, 'k': 100}]

In [11]:
best_ndcg = -np.inf
train_scores, valid_scores = [], []

model = VAE(**model_kwargs).to(device)
model_best = VAE(**model_kwargs).to(device)

In [12]:
learning_kwargs = {
    'model': model,
    'train_data': train_data,
    'batch_size': args.batch_size,
    'beta': args.beta,
    'gamma': args.gamma
}

In [13]:
decoder_params = set(model.decoder.parameters())
encoder_params = set(model.encoder.parameters())

optimizer_encoder = optim.Adam(encoder_params, lr=args.lr)
optimizer_decoder = optim.Adam(decoder_params, lr=args.lr)


In [14]:
for epoch in range(args.n_epochs):

    if args.not_alternating:
        run(opts=[optimizer_encoder, optimizer_decoder], n_epochs=1, dropout_rate=0.5, **learning_kwargs)
    else:
        run(opts=[optimizer_encoder], n_epochs=args.n_enc_epochs, dropout_rate=0.5, **learning_kwargs)
        model.update_prior()
        run(opts=[optimizer_decoder], n_epochs=args.n_dec_epochs, dropout_rate=0, **learning_kwargs)

    train_scores.append(
        evaluate(model, train_data, train_data, metrics, 0.01)[0]
    )
    valid_scores.append(
        evaluate(model, valid_in_data, valid_out_data, metrics, 1)[0]
    )
    
    if valid_scores[-1] > best_ndcg:
        best_ndcg = valid_scores[-1]
        model_best.load_state_dict(deepcopy(model.state_dict()))
        

    print(f'epoch {epoch} | valid ndcg@100: {valid_scores[-1]:.4f} | ' +
          f'best valid: {best_ndcg:.4f} | train ndcg@100: {train_scores[-1]:.4f}')

epoch 0 | valid ndcg@100: 0.0781 | best valid: 0.0781 | train ndcg@100: 0.1207
epoch 1 | valid ndcg@100: 0.1005 | best valid: 0.1005 | train ndcg@100: 0.1433
epoch 2 | valid ndcg@100: 0.1106 | best valid: 0.1106 | train ndcg@100: 0.1544
epoch 3 | valid ndcg@100: 0.1186 | best valid: 0.1186 | train ndcg@100: 0.1593
epoch 4 | valid ndcg@100: 0.1207 | best valid: 0.1207 | train ndcg@100: 0.1627
epoch 5 | valid ndcg@100: 0.1240 | best valid: 0.1240 | train ndcg@100: 0.1650
epoch 6 | valid ndcg@100: 0.1259 | best valid: 0.1259 | train ndcg@100: 0.1669
epoch 7 | valid ndcg@100: 0.1269 | best valid: 0.1269 | train ndcg@100: 0.1685
epoch 8 | valid ndcg@100: 0.1277 | best valid: 0.1277 | train ndcg@100: 0.1699
epoch 9 | valid ndcg@100: 0.1288 | best valid: 0.1288 | train ndcg@100: 0.1713
epoch 10 | valid ndcg@100: 0.1315 | best valid: 0.1315 | train ndcg@100: 0.1724
epoch 11 | valid ndcg@100: 0.1316 | best valid: 0.1316 | train ndcg@100: 0.1735
epoch 12 | valid ndcg@100: 0.1313 | best valid: 0.

In [15]:
test_metrics = [{'metric': ndcg, 'k': 100}, {'metric': recall, 'k': 20}, {'metric': recall, 'k': 50}]

final_scores = evaluate(model_best, test_in_data, test_out_data, test_metrics)

for metric, score in zip(test_metrics, final_scores):
    print(f"{metric['metric'].__name__}@{metric['k']}:\t{score:.4f}")

ndcg@100:	0.0932
recall@20:	0.0751
recall@50:	0.0854


In [17]:
torch.save(model_best, "RecVAE_50_epoch")